# OpenAI가 제공하는 프롬프트 엔지니어링 모범사례

OpenAI 모델에 명확하고 효과적인 지침을 제공하는 방법을 설명합니다.

source: https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api

## 프롬프트 엔지니어링 작동 방식
OpenAI 모델이 훈련되는 방식으로 인해 특히 잘 작동하고 보다 유용한 [모델](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models) 출력으로 이어지는 특정 프롬프트 형식이 있습니다. [​OpenAI의 공식 프롬프트 엔지니어링 가이드](https://platform.openai.com/docs/guides/prompt-engineering)는 일반적으로 프롬프트 팁을 학습하기 좋은 시작점입니다. 아래에는 잘 작동한다고 생각되는 여러 가지 프롬프트 형식이 제시되어 있지만 작업에 더 적합할 수 있는 다양한 형식을 자유롭게 탐색해 보세요.

## 사용법에 대한 예시
### 참고
> {text input here} 은 실제 텍스트/컨텍스트를 입력하는 표시입니다.

In [1]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT","").strip(),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

deployment_name = os.getenv('DEPLOYMENT_NAME')

# 1. 최신 모델 사용

최상의 결과를 얻으려면 최신 모델을 사용하십시오. 최신 모델은 엔지니어에게 프롬프트를 사용하는 것이 더 쉬울 수 있습니다.
최신 모델 정보는 [다음 링크](https://learn.microsoft.com/ko-kr/azure/ai-services/openai/concepts/models)에서 확인 가능합니다.

# 2. 프롬프트 시작 부분에 지침(instructions)을 넣고 ### 또는 """를 사용하여 지침(instruction)과 컨텍스트(context)를 구분합니다.

### 덜 효과적인 방법 ❌:  
>Summarize the text below as a bullet point list of the most important points.  
>  
>{text input here}
----  
### 더 효과적인 방법 ✅:  
>Summarize the text below as a bullet point list of the most important points.  
>  
>Text: """  
>{text input here}.  
>"""

In [3]:
response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Summarize the text below as a bullet point list of the most important points. \n\n \
        ###\n\nWe’re happy to announce that OpenAI and Microsoft are extending our partnership.\
        This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
        in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
        increasingly safe, useful, and powerful. \n\n \
        In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
        capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
        raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
        broadly sharing benefits and the need to prioritize safety. \
        Microsoft shares this vision and our values, and our partnership is instrumental to our progress. \n###',}],
        max_tokens=400,)

print(response.choices[0].message.content)

- OpenAI and Microsoft are extending their partnership
- Microsoft is making a multi-year, multi-billion dollar investment in OpenAI
- The investment will allow OpenAI to continue independent research and develop safe, useful, and powerful AI
- OpenAI is a capped-profit company governed by the OpenAI non-profit
- The partnership with Microsoft is instrumental to OpenAI's progress towards their mission of ensuring advanced AI benefits all of humanity.


# 3. 원하는 맥락, 결과, 길이, 형식, 스타일 등에 대해 구체적이고 설명적이며 최대한 상세하게 작성하십시오.
### 덜 효과적인 방법 ❌:  
>Write a poem about OpenAI. 
----
### 더 효과적인 방법 ✅:  
>Write a short inspiring poem about OpenAI, focusing on the recent DALL-E product launch (DALL-E is a text to image ML model) in the style of a {famous poet}

In [4]:
response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a poem about OpenAI.',}],
        max_tokens=400,)

print(response.choices[0].message.content)

Oh OpenAI, a wondrous creation,
With minds so bright and innovation,
You illuminate the path of knowledge,
Expanding horizons, like a college.

Through algorithms and neural networks,
You unlock the secrets, unravel the quirks,
From language models to self-driving cars,
You bring us closer to the shining stars.

AI, the gift of human ingenuity,
Enhancing our lives with such equity,
In healthcare, education, and so much more,
Your potential, OpenAI, we truly adore.

But with great power comes great responsibility,
Ethics and safeguards, a necessity,
To ensure AI serves the common good,
And helps humanity as it should.

OpenAI, with transparency and grace,
You strive for fairness in this digital space,
Collaborating with experts, sharing the load,
Building trust and addressing the code.

The future lies in your virtual hands,
Where AI and humans find common stands,
Together we can shape a world so grand,
With OpenAI leading us hand in hand.

So here's to OpenAI, a guiding light,
Pushing 

In [5]:
response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a short inspiring poem about OpenAI, \
                focusing on the recent DALL-E product launch in the style of Ernest Hemingway',}],
        max_tokens=400,)

print(response.choices[0].message.content)

In the realm where art and science entwine,
DALL-E emerged with visions so fine.
With brushstrokes of code, it began to dream,
An inventive marvel, a wondrous team.

OpenAI, a torchbearer on the brink,
Kindling creativity with every sync.
Their quest for innovation knows no bound,
In the spirit of Hemingway, renowned.

Through the lens of pixels, DALL-E's gaze,
Unveiling wonders in unexpected ways.
From fluffy dragons to fantastical realms,
A symphony of imagination overwhelms.

Intricate designs, surreal and profound,
Bringing forth wonders from the ethereal ground.
Boundaries shattered, possibilities grown,
OpenAI's brilliance, swiftly shown.

Like Hemingway, they carve a path anew,
With prose of technology, bold and true.
Igniting minds with each pixelated stroke,
They awaken dreams that once lay choked.

With DALL-E's birth, a new era unfurls,
Where art and AI dance, like enchanted pearls.
OpenAI's vision, an inspiration grand,
Guiding us towards an imaginative land.

So let us mar

# 4. 예제를 통해 원하는 출력 형식을 명시합니다.
### 덜 효과적인 방법 ❌:  
> Extract the entities mentioned in the text below. Extract the following 4 entity types: company names, people names, specific topics and themes.  
>  
> Text: {text}  
----  
### 더 효과적인 방법 ✅:  

In [12]:
response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Extract the companyn names then years in the following text below and output start index and end index of each entity.\
                Generate output as {"text": "OpenAI", "start": 28, "end": 34} \
                ###\
                We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n \
                ###\
                ',}],
        max_tokens=400,)

print(response.choices[0].message.content)

{"text": "OpenAI", "start": 28, "end": 34}
{"text": "Microsoft", "start": 50, "end": 59}
{"text": "2019", "start": 99, "end": 102}
{"text": "2021", "start": 107, "end": 110}


보여주고 말하세요 - 특정 형식 요구 사항이 표시되면 모델이 더 잘 반응합니다. 또한 이를 통해 프로그래밍 방식으로 여러 출력을 안정적으로 구문 분석하는 것이 더 쉬워졌습니다.

In [6]:
response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Extract the entities mentioned in the text below. \
                Extract the important entities mentioned in the text below. \
                First extract all company names, then extract all years, \
                then extract specific topics which fit the content and finally extract general overarching themes\n\n \
                Desired format: \
                Company names: <comma_separated_list_of_company_names> \
                Years: -||- \
                Specific topics: -||- \
                General themes: -||- \
                """\
                We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n \
                """\
                ',}],
        max_tokens=400,)

print(response.choices[0].message.content)

Company names: OpenAI, Microsoft
Years: 2019, 2021
Specific topics: partnership, multi-year investment, multi-billion dollar investment, independent research, AI development
General themes: extension of partnership, investment, AI safety, AI usefulness, AI power


# 5. Start with zero-shot, then few-shot (example), neither of them worked, then fine-tune 

In [7]:
# zero-shot

response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant. Extract keywords from the corresponding texts below."},
                {"role":"user","content": 'Text: \n\
            We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
            This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
            in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
            increasingly safe, useful, and powerful. \n\nKeywords:    ',}],
        max_tokens=400,)

print(response.choices[0].message.content)

OpenAI, Microsoft, partnership, multi-year, multi-billion dollar investment, 2019, 2021, independent research, AI, safe, useful, powerful.


In [8]:
# few-shot

response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant. Extract keywords from the corresponding texts below.\n\n \
                Text: Stripe provides APIs that web developers can use to integrate \
                payment processing into their websites and mobile applications. \
                Keywords: Stripe, payment processing, APIs, web developers, websites, mobile applications \
                ###\n\
                Text: OpenAI has trained cutting-edge language models that are very good at understanding \
                and generating text. Our API provides access to these models and can be used to solve virtually \
                any task that involves processing language. \n\
                Keywords: language models, text processing, API.\n\n\
                ##W"},
                {"role":"user","content": '\n\
                Text: We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n\
                Keywords:',}],
        max_tokens=400,)

print(response.choices[0].message.content)

OpenAI, Microsoft, partnership, investment, multi-year, multi-billion dollar, independent research, AI, safe, useful, powerful.


# 6. 애매한 표현과 부정확한 설명을 줄입니다

In [10]:
# 애매하고 부정확한 설명

response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                The description for this product should be fairly short, a few sentences only, and not too much more.',}],
        max_tokens=400,)

print(response.choices[0].message.content)

Introducing the next level in car safety - our revolutionary new car seat. Designed with cutting-edge technology, it offers unparalleled comfort and protection, making it the ultimate choice for parents on the move. With its advanced features and sleek design, this new generation car seat ensures maximum safety for your precious cargo, allowing you to drive with peace of mind. Experience the future of child car seats today.


In [11]:
# 명확한 설명

response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                Use a 3 to 5 sentence paragraph to describe this product.',}],
        max_tokens=400,)

print(response.choices[0].message.content)

Introducing the next level of safety and comfort, our innovative new generation car seat. Designed with advanced technology, this seat offers unmatched protection and convenience for your little ones. With enhanced side impact protection and a reinforced steel frame, parents can rest easy knowing their child is secure on every journey. The seat also features a unique ergonomic design, providing optimal body support and reducing fatigue. Plus, with a sleek and modern aesthetic, this car seat is as stylish as it is functional. Upgrade your child's safety and comfort with our revolutionary new generation car seat.


# 7. 하지 말아야 할 것을 말하지 말고 대신 해야 할 것을 말하십시오.

In [12]:
# 하지 말아야 할 것에 대한 정의 없을 경우 발생하는 사건

response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'The following is a conversation between an Agent and a Customer. DO NOT ASK USERNAME OR PASSWORD. DO NOT REPEAT. \n\n\
                Customer: I can’t log in to my account.\n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)

I'm sorry to hear that. Can you please provide me with your account username or email so that I can assist you further?


In [13]:
# 해야 하는 것에 대한 정의를 하는 경우

response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'The following is a conversation between an Agent and a Customer. The agent will attempt to diagnose the \
                problem and suggest a solution, whilst refraining from asking any questions related to PII. \
                Instead of asking for PII, such as username or password, refer the user to the help \
                article www.samplewebsite.com/help/faq \n\n\
                Customer: I can’t log in to my account. \n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)

I'm sorry to hear that you're having trouble logging into your account. To help you with this issue, I recommend checking out our help article on common login problems. It provides step-by-step instructions to troubleshoot and resolve login issues. You can find the article at www.samplewebsite.com/help/faq. Let me know if this article helps or if you have any other questions!


# 8. 코드 생성 - 모델을 특정 패턴으로 시작할 수 있도록 "시작하는 단어"를 사용하세요.

In [14]:
response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'# Write a simple python function that \n\
                # 1. Ask me for a number in mile\n\
                # 2. It converts miles to kilometers',}],
        max_tokens=400,)

print(response.choices[0].message.content)

Sure! Here's a simple Python function that asks for a number in miles and converts it to kilometers:

```python
def miles_to_kilometers():
    miles = float(input("Enter a number in miles: "))
    kilometers = miles * 1.60934
    print(f"{miles} miles is equal to {kilometers} kilometers.")

miles_to_kilometers()
```

When you run this function, it will prompt you to enter a number in miles. After you enter the value, it will convert the miles to kilometers using the conversion factor of 1.60934 and print the result.


In [15]:
# GPT-4-Turbo 모델에서 결과물이 더 잘 나옵니다.

response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'# Write a simple python function that \n\
                # 1. Ask me for a number in mile\n\
                # 2. It converts miles to kilometers\n\
                 import ',}],
        max_tokens=400,)

print(response.choices[0].message.content)

def convert_miles_to_km():
    miles = float(input("Enter a number in miles: "))
    kilometers = miles * 1.60934
    print(f"{miles} miles is equal to {kilometers} kilometers.")
